# BERT Training

Single GPU BERT Training Notebook

In [1]:
# GPU selection
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from pathlib import Path
import pickle
import multiprocessing
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.compat.v1.keras import layers

import utils
import bert_utils
import bert_optimizer

## Notebook Parameters

In [3]:
BERTLARGE     = False
USE_AMP       = True
USE_XLA       = True
MAX_SEQ_LEN   = 128
LEARNING_RATE = 1e-5
TUNE_LAYERS   = -1

In [4]:
if BERTLARGE:
    BERT_PATH = "https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1"
    H_SIZE = 1024
else:
    BERT_PATH = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
    H_SIZE = 768

## Create TensorFlow session

In [5]:
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"
os.environ["TF_GPU_THREAD_COUNT"] = "2"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.Experimental.use_numa_affinity = True
if USE_XLA:
    opt_level = tf.OptimizerOptions.ON_1
    tf.enable_resource_variables()
else:
    opt_level = tf.OptimizerOptions.OFF
config.graph_options.optimizer_options.global_jit_level = opt_level
config.graph_options.rewrite_options.auto_mixed_precision = USE_AMP
config.intra_op_parallelism_threads = multiprocessing.cpu_count()//2
config.inter_op_parallelism_threads = multiprocessing.cpu_count()//2
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

## Load Dataset

### Create Tokenizer

In [6]:
tokenizer = bert_utils.create_tokenizer_from_hub_module(BERT_PATH, sess)

### Preprocess Data

In [7]:
train_text, train_label, num_classes = utils.load_ag_news_dataset(max_seq_len=MAX_SEQ_LEN,
                                                                  test=False)

train_label = np.asarray(train_label)
train_examples = bert_utils.convert_text_to_examples(train_text, train_label)
feat = bert_utils.convert_examples_to_features(tokenizer,
                                               train_examples,
                                               max_seq_length=MAX_SEQ_LEN,
                                               verbose=True)

(train_input_ids, train_input_masks, train_segment_ids, train_labels) = feat

train_input_ids, train_input_masks, train_segment_ids, train_labels = shuffle(train_input_ids,
                                                                              train_input_masks,
                                                                              train_segment_ids,
                                                                              train_labels)

Loaded training set from: /home/jovyan/.keras/datasets/ag_news
Examples: 120000 Classes: 4


Converting examples to features: 100%|██████████| 120000/120000 [01:39<00:00, 1207.65it/s]


In [8]:
examples, labels, num_classes = utils.load_ag_news_dataset(max_seq_len=MAX_SEQ_LEN,
                                                           test=True)
labels = np.asarray(labels)
test_examples = bert_utils.convert_text_to_examples(examples, labels)
feat = bert_utils.convert_examples_to_features(tokenizer,
                                               test_examples,
                                               max_seq_length=MAX_SEQ_LEN,
                                               verbose=True)

(test_input_ids, test_input_masks, test_segment_ids, test_labels) = feat

test_input_ids, test_input_masks, test_segment_ids, test_labels = shuffle(test_input_ids,
                                                                          test_input_masks,
                                                                          test_segment_ids,
                                                                          test_labels)

test_set = ([test_input_ids, test_input_masks, test_segment_ids], test_labels)

Converting examples to features:   2%|▏         | 128/7600 [00:00<00:05, 1277.54it/s]

Loaded test set from: /home/jovyan/.keras/datasets/ag_news
Examples: 7600 Classes: 4


Converting examples to features: 100%|██████████| 7600/7600 [00:06<00:00, 1220.48it/s]


## Build Keras Model

In [9]:
if USE_AMP:
    tf.keras.mixed_precision.experimental.set_policy('infer_float32_vars')

in_id = layers.Input(shape=(MAX_SEQ_LEN,), name="input_ids")
in_mask = layers.Input(shape=(MAX_SEQ_LEN,), name="input_masks")
in_segment = layers.Input(shape=(MAX_SEQ_LEN,), name="segment_ids")

in_bert = [in_id, in_mask, in_segment]

l_bert = bert_utils.BERT(fine_tune_layers=TUNE_LAYERS,
                         bert_path=BERT_PATH,
                         return_sequence=False,
                         output_size=H_SIZE,
                         debug=False)(in_bert)

out_pred = layers.Dense(num_classes, activation="softmax")(l_bert)

model = tf.keras.models.Model(inputs=in_bert, outputs=out_pred)

In [10]:
opt = tf.keras.optimizers.Adam(lr=LEARNING_RATE)

if USE_AMP:
    opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt, "dynamic")

In [11]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (BERT)                     (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]            

## Train Model

In [13]:
log = model.fit([train_input_ids, train_input_masks, train_segment_ids],
                train_labels, validation_data=test_set,
                workers=4, use_multiprocessing=True,
                verbose=1, callbacks=[],
                epochs=1, batch_size=96)

Train on 120000 samples, validate on 7600 samples


/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


120000/120000 [==============================] - 269s 2ms/sample - loss: 0.2251 - acc: 0.9227 - val_loss: 0.1780 - val_acc: 0.9389


In [14]:
model.evaluate([test_input_ids, test_input_masks, test_segment_ids], test_labels, batch_size=1)

7600/7600 [==============================] - 63s 8ms/sample - loss: 0.1780 - acc: 0.9389


[0.1780370040601778, 0.9389474]

# Save Weights

In [17]:
tf.keras.backend.set_learning_phase(0)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

eval_graph_def = sess.graph.as_graph_def()

frozen_graph_def = tf.graph_util.convert_variables_to_constants(
    sess,
    eval_graph_def,
    [model.output.op.name]
)

frozen_graph_def = tf.graph_util.remove_training_nodes(frozen_graph_def)

with open('frozen_model.pb', 'wb') as f:
    f.write(frozen_graph_def.SerializeToString())